In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Alter dataset 1 to be on correct format for training model:

In [2]:
folder_path = "../data/dataset_1"

def read_and_combine_csv_files(folder_path):
    data_frames = []

    for file in os.listdir(folder_path):
        if file.lower().endswith(".csv"):
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path)
            data_frames.append(df)

    combined_df = pd.concat(data_frames, ignore_index=True)
    return combined_df

data_1 = read_and_combine_csv_files(folder_path)

In [3]:
data_1.shape

(61614, 21)

In [4]:
data_1.isnull().sum()

Province_State              0
Country_Region              0
Last_Update                19
Lat                      2142
Long_                    2142
Confirmed                   0
Deaths                      0
Recovered               46492
Active                  46492
FIPS                       19
Incident_Rate            2142
Total_Test_Results      24977
People_Hospitalized     56485
Case_Fatality_Ratio     12587
UID                         0
ISO3                        0
Testing_Rate            15693
Hospitalization_Rate    56485
Date                     9860
People_Tested           49798
Mortality_Rate          49587
dtype: int64

In [5]:
data_1.tail(5)

Province_State Country_Region          Last_Update      Lat     Long_  \
61609       Virginia             US  2021-06-15 04:31:17  37.7693  -78.1700   
61610     Washington             US  2021-06-15 04:31:17  47.4009 -121.4905   
61611  West Virginia             US  2021-06-15 04:31:17  38.4912  -80.9545   
61612      Wisconsin             US  2021-06-15 04:31:17  44.2685  -89.6165   
61613        Wyoming             US  2021-06-15 04:31:17  42.7560 -107.3025   

       Confirmed  Deaths  Recovered  Active  FIPS  ...  Total_Test_Results  \
61609     677812   11318        NaN     NaN  51.0  ...           7553224.0   
61610     444722    5782        NaN     NaN  53.0  ...           7358062.0   
61611     163144    2853        NaN     NaN  54.0  ...           2972502.0   
61612     676276    8015        NaN     NaN  55.0  ...           8893572.0   
61613      61234     725        NaN     NaN  56.0  ...            770872.0   

       People_Hospitalized  Case_Fatality_Ratio         UID  ISO3  \
61609                  NaN             1.669785  84000051.0   USA   
61610                  NaN             1.300138  84000053.0   USA   
61611                  NaN             1.748762  84000054.0   USA   
61612                  NaN             1.185167  84000055.0   USA   
61613                  NaN             1.183983  84000056.0   USA   

        Testing_Rate  Hospitalization_Rate        Date People_Tested  \
61609   88491.678128                   NaN  2021-06-14           NaN   
61610   96627.253988                   NaN  2021-06-14           NaN   
61611  165862.621760                   NaN  2021-06-14           NaN   
61612       0.000015                   NaN  2021-06-14           NaN   
61613  133193.954651                   NaN  2021-06-14           NaN   

       Mortality_Rate  
61609             NaN  
61610             NaN  
61611             NaN  
61612             NaN  
61613             NaN  

[5 rows x 21 columns]

In [6]:
data_1 = data_1[["Date", "Province_State", "Confirmed", "Deaths", "Active", "Testing_Rate", "Hospitalization_Rate"]]
# Testing_Rate = number of tests taken per 100 000 people
# Hospitalization_Rate = Total number hospitalized / Number cases. Note: only have data until Aug 31
data_1.head(5)

Date  Province_State  Confirmed  Deaths  Active   Testing_Rate  \
0  2022-01-21         Alabama    1120881   16824     NaN            NaN   
1  2022-01-21          Alaska     195345    1051     NaN  511483.777485   
2  2022-01-21  American Samoa         18       0     NaN    3846.084722   
3  2022-01-21         Arizona    1701949   25502     NaN  230705.356452   
4  2022-01-21        Arkansas     726716    9484     NaN  156042.804635   

   Hospitalization_Rate  
0                   NaN  
1                   NaN  
2                   NaN  
3                   NaN  
4                   NaN

In [7]:
data_1["Province_State"].value_counts()

Alabama                     1062
Puerto Rico                 1062
Nevada                      1062
New Hampshire               1062
New Jersey                  1062
New Mexico                  1062
New York                    1062
North Carolina              1062
North Dakota                1062
Northern Mariana Islands    1062
Ohio                        1062
Oklahoma                    1062
Oregon                      1062
Pennsylvania                1062
Rhode Island                1062
Montana                     1062
South Carolina              1062
South Dakota                1062
Tennessee                   1062
Texas                       1062
Utah                        1062
Vermont                     1062
Virgin Islands              1062
Virginia                    1062
Washington                  1062
West Virginia               1062
Wisconsin                   1062
Wyoming                     1062
Nebraska                    1062
Missouri                    1062
Alaska    

In [8]:
# Remove "states" that are not actually states
data_1 = data_1.loc[
    (data_1["Province_State"] != "Recovered")
    & (data_1["Province_State"] != "Diamond Princess")
    & (data_1["Province_State"] != "Grand Princess")
]

data_1["Province_State"].unique()

array(['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Guam', 'Hawaii',
       'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
       'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan',
       'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska',
       'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Northern Mariana Islands',
       'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virgin Islands', 'Virginia',
       'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype=object)

In [9]:
# Convert Date-field to datetime 
data_1['Date'] = pd.to_datetime(data_1['Date'])
print(type(data_1.Date[0]))
# Sort data by Date
data_1 = data_1.sort_values(by="Date")
data_1

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


Date Province_State  Confirmed  Deaths  Active  Testing_Rate  \
20721 2020-04-12    Connecticut      12035     554     NaN  1.156148e+03   
20754 2020-04-12           Ohio       6604     363     NaN  5.973514e-08   
20755 2020-04-12       Oklahoma       1970      96  1009.0  6.222602e+02   
20756 2020-04-12         Oregon       1527      52     NaN  7.426366e+02   
20757 2020-04-12   Pennsylvania      23219     572     NaN  9.921750e+02   
...          ...            ...        ...     ...     ...           ...   
60737        NaT       Virginia    2107801   22077     NaN           NaN   
60738        NaT     Washington    1830508   14493     NaN           NaN   
60739        NaT  West Virginia     605322    7479     NaN           NaN   
60740        NaT      Wisconsin    1885519   15379     NaN           NaN   
60741        NaT        Wyoming     178032    1906     NaN           NaN   

       Hospitalization_Rate  
20721             13.743249  
20754             29.497274  
20755             22.639594  
20756             23.510151  
20757              9.142035  
...                     ...  
60737                   NaN  
60738                   NaN  
60739                   NaN  
60740                   NaN  
60741                   NaN  

[59472 rows x 7 columns]

In [10]:
# Remove all rows with missing Date field
data_1 = data_1.dropna(subset=["Date"])
data_1

Date Province_State  Confirmed  Deaths  Active  Testing_Rate  \
20721 2020-04-12    Connecticut      12035     554     NaN  1.156148e+03   
20754 2020-04-12           Ohio       6604     363     NaN  5.973514e-08   
20755 2020-04-12       Oklahoma       1970      96  1009.0  6.222602e+02   
20756 2020-04-12         Oregon       1527      52     NaN  7.426366e+02   
20757 2020-04-12   Pennsylvania      23219     572     NaN  9.921750e+02   
...          ...            ...        ...     ...     ...           ...   
41463 2022-09-20     New Mexico     616360    8524     NaN  3.762336e+05   
41462 2022-09-20     New Jersey    2715620   34689     NaN  3.566158e+05   
41461 2022-09-20  New Hampshire     347953    2687     NaN           NaN   
41472 2022-09-20    Puerto Rico     948106    5077     NaN  1.449366e+05   
41429 2022-09-20         Alaska     296805    1364     NaN           NaN   

       Hospitalization_Rate  
20721             13.743249  
20754             29.497274  
20755             22.639594  
20756             23.510151  
20757              9.142035  
...                     ...  
41463                   NaN  
41462                   NaN  
41461                   NaN  
41472                   NaN  
41429                   NaN  

[49952 rows x 7 columns]

In [11]:
data_1.loc[data_1["Province_State"] == "Virginia"]

Date Province_State  Confirmed  Deaths  Active   Testing_Rate  \
20768 2020-04-12       Virginia       5274     141     NaN     505.676450   
20885 2020-04-13       Virginia       5747     149     NaN     523.584112   
12991 2020-04-14       Virginia       6182     154  5307.0     540.808855   
12874 2020-04-15       Virginia       6500     195  5553.0     558.590050   
33472 2020-04-16       Virginia       6889     208  5730.0     587.361187   
...          ...            ...        ...     ...     ...            ...   
1445  2022-09-16       Virginia    2070182   21679     NaN  174279.232464   
1503  2022-09-17       Virginia    2070182   21679     NaN  174407.437907   
47629 2022-09-18       Virginia    2070182   21679     NaN  174407.437907   
47745 2022-09-19       Virginia    2074003   21693     NaN  174407.437907   
41481 2022-09-20       Virginia    2075283   21712     NaN  174702.791945   

       Hospitalization_Rate  
20768             14.239666  
20885             21.541674  
12991             20.737625  
12874             19.969231  
33472             19.407751  
...                     ...  
1445                    NaN  
1503                    NaN  
47629                   NaN  
47745                   NaN  
41481                   NaN  

[892 rows x 7 columns]

In [12]:
# Finally, reset index
data_1 = data_1.reset_index().drop(columns=["index"])
data_1.head(5)

Date Province_State  Confirmed  Deaths  Active  Testing_Rate  \
0 2020-04-12    Connecticut      12035     554     NaN  1.156148e+03   
1 2020-04-12           Ohio       6604     363     NaN  5.973514e-08   
2 2020-04-12       Oklahoma       1970      96  1009.0  6.222602e+02   
3 2020-04-12         Oregon       1527      52     NaN  7.426366e+02   
4 2020-04-12   Pennsylvania      23219     572     NaN  9.921750e+02   

   Hospitalization_Rate  
0             13.743249  
1             29.497274  
2             22.639594  
3             23.510151  
4              9.142035

In [13]:
# Write dataset to file
data_1.to_csv('../data/modeling.csv', index=False)